# Розширення генерації на основі пошуку (RAG) та векторні бази даних

In [1]:
%pip install getenv openai faiss-cpu pandas numpy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 38.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pandas as pd
import numpy as np
import faiss

from dotenv import load_dotenv

load_dotenv()

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## Створення нашої бази знань

Налаштування FAISS для векторного пошуку


In [5]:
# Шляхи до файлів (даних для RAG)
data_paths = [
    "data/frameworks.md", 
    "data/own_framework.md", 
    "data/perceptron.md"
] 

# Ініціалізація порожнього DataFrame
df = pd.DataFrame(columns=['path', 'text'])

# Сучасний спосіб додавання рядків до DataFrame
for path in data_paths:
    try:
        with open(path, 'r', encoding='utf-8') as file:
            file_content = file.read()
        
        # Використовуємо concat замість застарілого append
        new_row = pd.DataFrame({'path': [path], 'text': [file_content]})
        df = pd.concat([df, new_row], ignore_index=True)
    except FileNotFoundError:
        print(f"Файл не знайдено: {path}")

df.head()

NameError: name 'pd' is not defined

In [6]:
def split_text(text, max_length, min_length):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(' '.join(current_chunk)) < max_length and len(' '.join(current_chunk)) > min_length:
            chunks.append(' '.join(current_chunk))
            current_chunk = []

    # Якщо останній фрагмент не досягнув мінімальної довжини, все одно додати його
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Припускаючи, що analyzed_df - це pandas DataFrame, а 'output_content' - це стовпець у цьому DataFrame
splitted_df = df.copy()
splitted_df['chunks'] = splitted_df['text'].apply(lambda x: split_text(x, 400, 300))

splitted_df

NameError: name 'df' is not defined

In [7]:
# Припускаючи, що 'chunks' - це стовпець списків у DataFrame splitted_df, ми розділимо фрагменти на різні рядки
flattened_df = splitted_df.explode('chunks')

flattened_df.head()

NameError: name 'splitted_df' is not defined

## Перетворення тексту на ембедінги

In [8]:
from azure.ai.inference import EmbeddingsClient
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
token = os.environ["GITHUB_TOKEN"]

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

def create_embeddings(text):
    """
    Створює ембедінги для тексту.
    
    Args:
        text: Текст або список текстів для ембедінгу
        model: Модель для ембедінгу (за замовчуванням mistral-embed)
        
    Returns:
        Вектор ембедінгу
    """
    # Обробка pandas Series
    if isinstance(text, pd.Series):
        # Беремо перший елемент з Series
        text = text.iloc[0]
    
    # Перетворюємо в список рядків для API
    if not isinstance(text, list):
        text = [str(text)]
    else:
        text = [str(item) for item in text]
    
    embeddings_response = embed_client.embed(
        input=text,
        model=embed_model_name
    )
    
    # Повернення ембедінгу для першого елемента
    return embeddings_response.data[0].embedding

# Приклад використання:
embeddings = create_embeddings(flattened_df['chunks'][0])

NameError: name 'flattened_df' is not defined

# Розширення генерації на основі пошуку (RAG) та векторні бази даних

In [9]:
cat = create_embeddings("cat")

cat

NameError: name 'pd' is not defined

In [10]:
import pickle
from pathlib import Path

def save_embeddings(df, folder="embeddings", filename="flattened_df.pkl"):
    """
    Зберігає DataFrame з ембедінгами в указану папку.
    
    Args:
        df: DataFrame з ембедінгами
        folder: Назва папки для збереження
        filename: Ім'я файлу для збереження
    """
    # Створення директорії, якщо вона не існує
    Path(folder).mkdir(parents=True, exist_ok=True)
    
    # Шлях до файлу
    file_path = os.path.join(folder, filename)
    
    # Збереження DataFrame
    with open(file_path, 'wb') as f:
        pickle.dump(df, f)
    
    print(f"DataFrame успішно збережено в {file_path}")

def load_embeddings(folder="embeddings", filename="flattened_df.pkl"):
    """
    Завантажує DataFrame з ембедінгами з указаної папки.
    
    Args:
        folder: Назва папки для завантаження
        filename: Ім'я файлу для завантаження
        
    Returns:
        DataFrame з ембедінгами або None, якщо файл не існує
    """
    # Шлях до файлу
    file_path = os.path.join(folder, filename)
    
    # Перевірка існування файлу
    if os.path.exists(file_path):
        # Завантаження DataFrame
        with open(file_path, 'rb') as f:
            df = pickle.load(f)
        
        print(f"DataFrame успішно завантажено з {file_path}")
        return df
    else:
        print(f"Файл {file_path} не знайдено")
        return None

def get_or_create_embeddings(df, chunk_column, embedding_function, folder="embeddings", filename="flattened_df.pkl"):
    """
    Завантажує DataFrame з ембедінгами або створює новий.
    
    Args:
        df: Вихідний DataFrame з текстами
        chunk_column: Назва стовпця з текстовими фрагментами
        embedding_function: Функція для створення ембедінгів
        folder: Назва папки для збереження/завантаження
        filename: Ім'я файлу для збереження/завантаження
        
    Returns:
        DataFrame з ембедінгами
    """
    # Спроба завантажити DataFrame
    loaded_df = load_embeddings(folder, filename)
    
    if loaded_df is not None:
        return loaded_df
    
    # Якщо завантаження не вдалося, створюємо ембедінги
    print("Створення нових ембедінгів...")
    
    # Створення ембедінгів
    embeddings = []
    for chunk in df[chunk_column]:
        embeddings.append(embedding_function(chunk))
    
    # Збереження ембедінгів в DataFrame
    df['embeddings'] = embeddings
    
    # Збереження DataFrame
    save_embeddings(df, folder, filename)
    
    return df


# Використовуємо функцію, яка розраховує ембедінги, 
# якщо вони не були раніше створені і збережені в папці в "15-rag-and-vector-databases/embeddings"
flattened_df = get_or_create_embeddings(
    splitted_df.explode('chunks'), 
    'chunks', 
    create_embeddings
)

flattened_df.head()

NameError: name 'splitted_df' is not defined

In [11]:
# Приклад використання:
compare_responses("Що таке перцептрон?")

NameError: name 'compare_responses' is not defined

# Пошук з використанням FAISS

Векторний пошук та схожість між нашим запитом і базою даних з використанням FAISS

### Створення індексу FAISS та підготовка до пошуку

In [12]:
# Отримуємо ембедінги як масив numpy
embeddings_list = flattened_df['embeddings'].to_list()
embeddings_array = np.array(embeddings_list).astype('float32')

# Визначаємо розмірність векторів
vector_dimension = len(embeddings_list[0])

# Створюємо індекс FAISS
index = faiss.IndexFlatL2(vector_dimension)  # L2 - це евклідова відстань

# Додаємо наші вектори до індексу
index.add(embeddings_array)

# Перевіряємо кількість векторів в індексі
print(f"Загальна кількість векторів в індексі: {index.ntotal}, розмірність векторів: {vector_dimension}")

NameError: name 'flattened_df' is not defined

In [13]:
# Ваше текстове запитання
question = "Що таке персептрон?"

# Перетворіть запитання у вектор запиту
query_vector = create_embeddings(question)

query_vector_array = np.array([query_vector]).astype('float32')

# Знайдіть найбільш схожі документи (k=5 - скільки найближчих сусідів шукаємо)
k = 5
distances, indices = index.search(query_vector_array, k)

# Виведіть найбільш схожі документи
for i in range(min(3, len(indices[0]))):
    idx = indices[0][i]
    print(f"Фрагмент {i+1}:")
    print(flattened_df['chunks'].iloc[idx])
    print(f"Шлях: {flattened_df['path'].iloc[idx]}")
    print(f"Відстань: {distances[0][i]}")
    print("-" * 50)

NameError: name 'pd' is not defined

## Поєднання всього для відповіді на запитання

In [14]:
from azure.ai.inference import ChatCompletionsClient

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

# Виберіть модель загального призначення для тексту
deployment = "gpt-4o-mini"

# Реалізація чатботів (при наявності і відсутності RAG)

In [15]:
def chatbot_with_rag(user_input):
    # Перетворіть запитання у вектор запиту
    query_vector = create_embeddings(user_input)
    query_vector_array = np.array([query_vector]).astype('float32')
    
    # Знайдіть найбільш схожі документи з FAISS
    k = 5  # кількість найближчих сусідів для пошуку
    distances, indices = index.search(query_vector_array, k)

    # додайте документи до запиту, щоб забезпечити контекст
    history = []
    for idx in indices[0]:
        history.append(flattened_df['chunks'].iloc[idx])

    # створюємо об'єкт повідомлення з контекстом
    context = "\n\n".join(history)  # всі знайдені фрагменти

    # Формуємо запит, що просить коротку, але завершену відповідь
    messages = [
        {"role": "system", "content": "You are an AI assistant that helps with AI questions. "},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {user_input}\n\n Provide a brief but complete answer based on the context. Answer in Ukrainian."}
    ]


    response = client.complete(
        temperature=0,
        model=deployment,
        messages=messages,
        max_tokens=300,
    )

    return response.choices[0].message.content


def chatbot_without_rag(user_input):
    """
    Чат-бот без використання RAG (прямий запит до моделі).
    """
    messages=[
        {"role": "system", "content": "You are an AI assistant that helps with AI questions. Provide brief but complete answers. Answer in Ukrainian."},
        {"role": "user", "content": f"Question: {user_input}"}
    ]

    response = client.complete(
        temperature=0,
        model=deployment,
        messages=messages,
        max_tokens=300,
    )

    return response.choices[0].message.content

# Порівняння відповідей RAG-системи

In [16]:
from IPython.display import display, Markdown, HTML

def compare_responses(user_input, save_to_file=False, filename="rag_comparison.md"):
    """
    Порівнює відповіді чат-боту з RAG та без RAG.
    
    Args:
        user_input: Запитання користувача
        save_to_file: Зберегти результат у файл Markdown
        filename: Назва файлу для збереження
    """
    # Отримання знайдених чанків
    query_vector = create_embeddings(user_input)

    query_vector_array = np.array([query_vector]).astype('float32')
    k = 5  # кількість найближчих сусідів для пошуку
    distances, indices = index.search(query_vector_array, k)
    
    # Отримання відповідей
    rag_response = chatbot_with_rag(user_input)
    no_rag_response = chatbot_without_rag(user_input)
    
    # Формування markdown-тексту
    markdown_text = f"""
# Порівняння відповідей

## 📝 Запит: {user_input}

## 🔍 Відповіді моделей

### Без використання RAG

{no_rag_response}

### З використанням RAG

{rag_response}

## 📚 Знайдені фрагменти тексту

"""
    
    # Додавання чанків
    for i, idx in enumerate(indices[0]):
        chunk_content = flattened_df['chunks'].iloc[idx]
        path = flattened_df['path'].iloc[idx]
        dist = float(distances[0][i])
        
        markdown_text += f"""
### Фрагмент {i+1} (відстань: {dist:.4f})

**Шлях**: {path}

{chunk_content}

"""
    
    # Виведення Markdown
    display(Markdown(markdown_text))
    
    # Для коректного відображення формул
    mathjax_script = """
    <script type="text/javascript">
        MathJax = {
            tex: {
                inlineMath: [['$', '$']]
            }
        };
    </script>
    <script type="text/javascript" id="MathJax-script" async
        src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-chtml.js">
    </script>
    """
    display(HTML(mathjax_script))
    
    # Збереження в файл, якщо потрібно
    if save_to_file:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(markdown_text)
        print(f"Результати збережено у файл: {filename}")


In [17]:
# Приклад використання:
compare_responses("Що таке перцептрон?")

NameError: name 'pd' is not defined

### Індивідуальне завдання

> Варіант 9 - Блокчейн технології

**Частина 1: Базова реалізація RAG-системи на власних даних**: Модифікуйте базовий ноутбук, щоб реалізувати RAG-систему для іншої тематики (згідно з вашим варіантом)

**Частина 2: Розширене завдання**: Розширте функціональність базового рішення, створивши інтерактивну систему для відповіді на запитання про навчальні курси. Така система матиме практичне застосування для навчальних закладів, освітніх платформ та онлайн-курсів

In [18]:
import os
import pandas as pd
import numpy as np
import faiss
import pickle
from pathlib import Path
from azure.ai.inference import EmbeddingsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv()

class BlockchainKnowledgeBase:
    def __init__(self, cache_file="blockchain_vectors_v9.pkl"):
        self.token = os.getenv("GITHUB_TOKEN")
        self.endpoint = "https://models.inference.ai.azure.com"
        self.model = "text-embedding-3-small"
        self.cache_file = cache_file
        self.client = EmbeddingsClient(endpoint=self.endpoint, credential=AzureKeyCredential(self.token))
        self.df = None
        self.index = None

    def get_embedding(self, text):
        """Генерація вектора для тексту."""
        response = self.client.embed(model=self.model, input=[str(text)])
        return response.data[0].embedding

    def build(self, file_list):
        """Завантаження файлів, розбиття на фрагменти та створення індексу."""
        if Path(self.cache_file).exists():
            print(f"📦 Завантаження бази знань з кешу: {self.cache_file}")
            with open(self.cache_file, "rb") as f:
                self.df = pickle.load(f)
        else:
            print("🚀 Створення нової бази знань з блокчейну...")
            raw_data = []
            for path in file_list:
                if os.path.exists(path):
                    with open(path, "r", encoding="utf-8") as f:
                        content = f.read()
                    # Просте розбиття на чанки по ~1500 символів
                    chunks = [content[i:i+1500] for i in range(0, len(content), 1200)]
                    for c in chunks:
                        raw_data.append({"path": path, "chunk": c})
            
            self.df = pd.DataFrame(raw_data)
            self.df["vector"] = self.df["chunk"].apply(self.get_embedding)
            with open(self.cache_file, "wb") as f:
                pickle.dump(self.df, f)

        # Створення FAISS індексу
        vectors = np.array(self.df["vector"].tolist()).astype('float32')
        self.index = faiss.IndexFlatL2(vectors.shape[1])
        self.index.add(vectors)
        print(f"✅ База готова. Оброблено {len(self.df)} фрагментів.")

# Шляхи до твоїх Markdown-файлів (переконайся, що папка data/blockchain_kb існує)
blockchain_files = [f"data/blockchain_kb/{i:02d}_Blockchain_Basics.md" for i in range(1, 11)] 
# Примітка: підстав реальні назви файлів зі свого датасету, якщо вони відрізняються

kb = BlockchainKnowledgeBase()
# kb.build(blockchain_files) # Розкоментуй після перевірки шляхів

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [19]:
%pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

def start_interactive_assistant():
    # Налаштування стилю
    style_header = "background: #1e293b; color: #38bdf8; padding: 15px; border-radius: 10px; border-bottom: 3px solid #38bdf8;"
    
    # Віджети
    user_input = widgets.Textarea(
        placeholder='Напишіть ваше питання про блокчейн тут...',
        layout=widgets.Layout(width='100%', height='100px')
    )
    
    mode_selector = widgets.ToggleButtons(
        options=[('Пошук курсів', 'find'), ('Питання (RAG)', 'ask'), ('Порівняння', 'compare')],
        description='Режим:',
        button_style='info'
    )
    
    run_btn = widgets.Button(
        description='🚀 Запитати асистента',
        button_style='primary',
        layout=widgets.Layout(width='100%', margin='10px 0px')
    )
    
    output_area = widgets.Output()

    def process_request(b):
        with output_area:
            clear_output()
            query = user_input.value.strip()
            if not query:
                print("⚠️ Будь ласка, введіть запит.")
                return
            
            print(f"🔍 Аналізую запит у режимі: {mode_selector.label}...")
            
            try:
                # Тут викликаються функції, які ти переписав раніше (chatbot_with_rag_blockchain тощо)
                if mode_selector.value == 'ask':
                    ans, src = chatbot_with_rag_blockchain(query)
                    display(Markdown(f"### 🤖 Відповідь:\n{ans}"))
                    display(Markdown(f"**📚 Джерела:** {', '.join(set(src))}"))
                elif mode_selector.value == 'find':
                    ans, courses = find_blockchain_course_by_topic(query)
                    display(Markdown(f"### 🎯 Рекомендовані курси:\n{ans}"))
                else:
                    # Режим порівняння
                    rag_ans, _ = chatbot_with_rag_blockchain(query)
                    no_rag_ans = chatbot_without_rag_blockchain(query)
                    display(Markdown(f"#### 🌐 Стандартна відповідь LLM:\n{no_rag_ans}"))
                    display(Markdown("---"))
                    display(Markdown(f"#### 🧠 Відповідь з власною базою знань:\n{rag_ans}"))
            except Exception as e:
                print(f"❌ Виникла помилка: {e}")

    run_btn.on_click(process_request)

    # Компонування інтерфейсу
    display(widgets.HTML(f"<div style='{style_header}'><h2>🧱 Blockchain Crypto-Expert v2.0</h2></div>"))
    display(widgets.VBox([mode_selector, user_input, run_btn, output_area]))

# Запуск
start_interactive_assistant()

HTML(value="<div style='background: #1e293b; color: #38bdf8; padding: 15px; border-radius: 10px; border-bottom…

In [21]:
educational_assistant_blockchain()

NameError: name 'educational_assistant_blockchain' is not defined